In [1]:
import os
import sys
import pickle
from flask import Flask, redirect, url_for, request, render_template, Response, jsonify, redirect
from flask_ngrok import run_with_ngrok
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from util import base64_to_pil
from PIL import Image, ImageOps
import numpy as np
import gc


# Declare a flask app
app = Flask(__name__)
run_with_ngrok(app)


#Predict Functions
def import_and_predict(test):
	models = load_model('/home/nivethjunnithan/Desktop/PlantDoctor/simple_classifier/Epoch-50-Val_acc1.00.hdf5')
	lbs=open(r'/home/nivethjunnithan/Desktop/PlantDoctor/simple_classifier/label_transform.pkl', 'rb')
	k=predi(test,models,lbs)
	if k == 'Banana':
		model=load_model('/home/nivethjunnithan/Desktop/PlantDoctor/banana_classifier/Epoch-45-Val_acc0.99.hdf5')
		lb=open(r'/home/nivethjunnithan/Desktop/PlantDoctor/banana_classifier/label_transform.pkl', 'rb')
		predic(test,lb,model)
	
        
	elif k == 'Paddy':
		model=load_model('/home/nivethjunnithan/Desktop/PlantDoctor/paddy_classifier/Epoch-05-Val_acc0.82.hdf5')
		lb=open(r'/home/nivethjunnithan/Desktop/PlantDoctor/paddy_classifier/label_transform.pkl', 'rb')
		predic(test,lb,model)

    
	elif k == 'Tomato':
		model=load_model('/home/nivethjunnithan/Desktop/PlantDoctor/tomato_classifier/Epoch-29-Val_acc0.99.hdf5')
		lb=open(r'/home/nivethjunnithan/Desktop/PlantDoctor/tomato_classifier/label_transform.pkl', 'rb')
		predic(test,lb,model)
        
	else :
		print (k)


def predic(test_image,labelencoder,mod):
    labeltransformer = pickle.load(labelencoder)
    test_image =image.img_to_array(test_image)
    test_image = np.array([test_image], dtype=np.float16) / 255.0
    prediction =mod.predict(test_image)
    print("Disease Classifier")
    print (prediction)
    print (np.max(prediction))
    print("\n")
    if(np.max(prediction)>0.5):
        print("Suitable")
        print(labeltransformer.inverse_transform(prediction))
    else :
        sim="Not a suitable type"

def predi(test_image,mod,labelencod):
    labeltransform = pickle.load(labelencod)
    test_image =image.img_to_array(test_image)
    test_image = np.array([test_image], dtype=np.float16) / 255.0
    prediction = mod.predict(test_image)
    print("Simple Classifier")
    print (prediction)
    print (np.max(prediction))
    print("\n")
    if(np.max(prediction)>0.75):
        sim= labeltransform.inverse_transform(prediction)
        print(sim)
    else :
        sim="Not a suitable type"
    
    return  sim

    
@app.route('/', methods=['GET'])
def index():
	return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def predict():
	if request.method == 'POST':
		try:
			size=(256,256)
			img = base64_to_pil(request.json)
			img = img.resize(size)
			result = import_and_predict(img)
			return jsonify(result=result)
		except ValueError:
			result = 'Image is not suitable. The image contains more than 3 input layers'
			return jsonify(result=result)
			
	return None

# Main function
if __name__ == '__main__':
	app.run()

Simple Classifier
[[3.4893351e-03 2.6812809e-11 9.9651068e-01]]
0.9965107


['Tomato']
Disease Classifier
[[9.9999595e-01 4.0541026e-06 8.0921387e-14]]
0.99999595


Suitable
['Tomato_Late_blight']
